## 1. セットアップ

### 2. パッケージインストール

In [ ]:
# 必要パッケージのインストール
!pip install -q -r ../requirements.txt

### 1. 環境確認

In [ ]:
# GPU確認
!nvidia-smi

import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Device name: {torch.cuda.get_device_name(0)}")

### 3. git

In [ ]:
!pip install nbstripout
!nbstripout --install

In [ ]:
!git config --global user.email "runpod@example.com"
!git config --global user.name "Runpod User"

In [ ]:
!git pull origin main

## 3. モジュールインポート

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import torch

# プロジェクトルートをパスに追加
sys.path.append('/workspace/atma_22_ca/')

from configs.config import *
from src.runner import Runner
from src.model_arcface import ModelArcFace
from src.util import Logger, Submission

print(f"Input dir: {DIR_INPUT}")
print(f"Model dir: {DIR_MODEL}")
print(f"Submission dir: {DIR_SUBMISSIONS}")

## 4. ロガー設定

In [ ]:
logger = Logger(path=DIR_LOG)

def get_run_name(model_type):
    run_name = model_type
    suffix = '_' + datetime.now().strftime("%Y%m%d%H%M")
    run_name = run_name + suffix
    return run_name

logger.info("Logger initialized")

## 5. データ読み込み

In [ ]:
df_train = pd.read_csv(FILE_TRAIN_META)
df_test = pd.read_csv(FILE_TEST_META)

# グループ化
# df_train['group'] = df_train['quarter'] + '_' + df_train['session'].astype(str)

# インデックス付与（前処理済み画像読み込み用）
df_train = df_train.reset_index(drop=False) 

## 6. パラメータ設定
### GPU最適化戦略
- batch_size: GPU VRAMに応じて動的調整
- num_workers: CPU並列化でデータローディング高速化
- ArcFace損失: 角度マージンでクラス間分離を強化

In [ ]:
memo = "ArcFace with EfficientNet-B0 backbone and post-processing optimization"
run_name = get_run_name("arcface_efficientnet_b0_postproc")
# run_name = 'arcface_efficientnet_b0_202512190737'
print(f"Run name: {run_name}")
print(f"Memo: {memo}")

In [ ]:
# デバイス確認とGPU最適化パラメータ設定
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")

# GPU・CPU仕様を確認
gpu_name = torch.cuda.get_device_name(0)
gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3  # GB
print(f"GPU: {gpu_name}, Memory: {gpu_memory:.1f}GB")
print(f"利用可能CPU: {os.cpu_count()} cores")

# 大容量GPU (20GB以上) の場合
batch_size = 128  # ArcFaceは少し控えめ（KNNより学習コスト高）
num_workers = 12
epochs = 10 

print(f"最適化パラメータ: batch_size={batch_size}, num_workers={num_workers}, epochs={epochs}")

In [ ]:
# ArcFaceモデルパラメータ
params = {
    # モデル設定
    'model_name': 'efficientnet_b0',  # バックボーン: 'efficientnet_b0', 'resnet50', etc.
    'embedding_dim': 1024,              # 埋め込みベクトル次元数
    'img_size': 224,                   # 入力画像サイズ
    'seed': 42,                       # ランダムシード
    # ArcFace固有パラメータ
    'arcface_s': 30.0,                 # スケールファクター（論文推奨値）
    'arcface_m': 0.5,                  # 角度マージン（0.5 rad ≈ 28.6度）
    # 訓練設定
    'batch_size': batch_size,          # GPU環境に応じて調整
    'epochs': epochs,                  # エポック数
    'lr': 1e-3,                        # 学習率
    'weight_decay': 1e-4,              # 重み減衰
    # EMA設定
    'use_ema': True,                   # EMA（Exponential Moving Average）を使用
    'ema_decay': 0.998,                # EMA減衰率
    # 推論設定
    'threshold': 0.5,                  # 未知選手判定の閾値
    # その他
    'num_workers': num_workers,        # DataLoaderのワーカー数
}

# CV設定（Validatorを明示的に作成）
from src.util import Validation
validator_normal = Validation.create_validator(
    method='stratified_group',
    n_splits=3,
    shuffle=True,
    random_state=42
)
cv_setting = {
    'validator': validator_normal,
    'group_col': 'quarter',  # グループ化列
}

## 7. Runner作成

In [ ]:
# Runnerの作成
runner = Runner(
    run_name=run_name,
    model_cls=ModelArcFace,
    params=params,
    df_train=df_train,
    df_test=df_test,
    cv_setting=cv_setting,
    logger=logger,
    seed=params.get('seed')
)

print(f"\nRunner created: {run_name}")
print(f"Model class: {ModelArcFace.__name__}")

## 8. CV学習

各Foldで以下を実行：
1. ArcFace損失で埋め込み学習（約20エポック）
2. EMAで重み平滑化
3. 訓練データからプロトタイプ（平均埋め込み）を計算
4. モデル・プロトタイプ保存

In [ ]:
%%time
# 3-fold CV学習
runner.train_cv()

## 9. CV評価

In [ ]:
scores_opt = runner.metric_cv_with_postprocessing(
    unknown_prob=0.1,
    max_labels=10
)

## 10. テストデータ予測

In [ ]:
pred_test = runner.predict_cv_with_postprocessing(
    unknown_prob=0.1,
    max_labels=10,
)

In [ ]:
# 予測分布の可視化
plt.figure(figsize=(10, 5))
pred_test['label_id'].value_counts().sort_index().plot(kind='bar')
plt.title('Test Prediction Distribution (ArcFace)')
plt.xlabel('Label ID')
plt.ylabel('Count')
plt.axhline(y=len(pred_test)/12, color='r', linestyle='--', alpha=0.5, label='Uniform')
plt.legend()    
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 11. 提出ファイル作成

In [ ]:
save_path = Submission.save(pred_test, run_name=run_name, logger=logger)

In [ ]:
pd.read_csv(save_path)

## 12. 閾値チューニング実験（高速）

学習済みモデルを使って異なる閾値で再予測

In [ ]:
# 閾値を変えて予測（学習済みモデル再利用）
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]
threshold_results = []

print("\n=== 閾値チューニング ===")
print("(学習済みモデルを再利用して高速実験)\n")

for thresh in thresholds:
    # パラメータ更新
    params_test = params.copy()
    params_test['threshold'] = thresh
    
    # 新しいRunnerで予測のみ
    runner_test = Runner(
        run_name=run_name,  # 同じrun_nameでモデル読み込み
        model_cls=ModelArcFace,
        params=params_test,
        df_train=df_train,
        df_test=df_test,
        cv_setting=cv_setting,
        logger=logger
    )
    
    # 予測のみ実行
    pred = runner_test.predict_cv()
    
    # 統計情報
    unknown_count = (pred['label_id'] == -1).sum()
    unknown_ratio = unknown_count / len(pred) * 100
    
    threshold_results.append({
        'threshold': thresh,
        'unknown_count': unknown_count,
        'unknown_ratio': unknown_ratio
    })
    
    print(f"Threshold {thresh:.1f}: unknown={unknown_count:4d} ({unknown_ratio:5.2f}%)")

df_threshold_results = pd.DataFrame(threshold_results)
print("\n=== 閾値チューニング結果 ===")
print(df_threshold_results)

In [ ]:
# 閾値とunknown比率の関係を可視化
if len(threshold_results) > 0:
    plt.figure(figsize=(10, 5))
    plt.plot(df_threshold_results['threshold'], df_threshold_results['unknown_ratio'], marker='o', linewidth=2)
    plt.xlabel('Threshold', fontsize=12)
    plt.ylabel('Unknown Ratio (%)', fontsize=12)
    plt.title('ArcFace: Effect of Threshold on Unknown Predictions', fontsize=14)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
print("\n※ 閾値が低い → unknown予測が増える（保守的）")
print("※ 閾値が高い → unknown予測が減る（積極的）")